In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from file_exchange import config
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
import gevent
from gevent.pool import Group
from gevent.select import select
from gevent.event import AsyncResult
import time
from datetime import datetime

# Parallel users

In [2]:
conf = config.default
conf['n_users'] = 3
conf

{'ack_size': 0,
 'bandwidth': 10,
 'chunk_max_size': 2,
 'file_size': 11,
 'gossip_size': 20,
 'header_size': 0.1,
 'layer_threshold': 0.001,
 'minimum_node_availability': 0,
 'n_file_chunks': 10,
 'n_layers': 3,
 'n_rounds': 2,
 'n_users': 3,
 'period': datetime.timedelta(0, 1)}

In [5]:
address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)

def output_view(view, conf):
    i = 0
    while True:
        print("Iteration #{} at time {}".format(i, datetime.now().time()))
        print(view.get())
        gevent.sleep(conf['period'].total_seconds())
        i+=1

#output_thread = gevent.spawn(output_view, address_book, conf)

for u in users:
    # Starts greenlet
    u.start()
gevent.joinall(users)

#gevent.kill(output_thread)


Lavette says Hello
Lavette started his/her devices
[update_state Lavette]Before lock
[update_state Lavette]After lock
User Lavette is at round #1/5
[update_state Lavette]After release
Annabel says Hello
Annabel started his/her devices
[update_state Annabel]Before lock
[update_state Annabel]After lock
User Annabel is at round #1/5
[update_state Annabel]After release
Demetra says Hello
Demetra started his/her devices
[update_state Demetra]Before lock
[update_state Demetra]After lock
User Demetra is at round #1/5
[update_state Demetra]After release
Buffy says Hello
Buffy started his/her devices
[update_state Buffy]Before lock
[update_state Buffy]After lock
User Buffy is at round #1/5
[update_state Buffy]After release
Moira says Hello
Moira started his/her devices
[update_state Moira]Before lock
[update_state Moira]After lock
User Moira is at round #1/5
[update_state Moira]After release
Mi says Hello
Mi started his/her devices
[update_state Mi]Before lock
[update_state Mi]After lock
User M

[update_state Lavette]Before lock
[update_state Lavette]After lock
User Lavette is at round #4/5
[update_state Lavette]After release
[update_state Annabel]Before lock
[update_state Annabel]After lock
User Annabel is at round #4/5
[update_state Annabel]After release
[update_state Demetra]Before lock
[update_state Demetra]After lock
User Demetra is at round #4/5
[update_state Demetra]After release
[update_state Buffy]Before lock
[update_state Buffy]After lock
User Buffy is at round #4/5
[update_state Buffy]After release
[update_state Moira]Before lock
[update_state Moira]After lock
User Moira is at round #4/5
[update_state Moira]After release
[update_state Mi]Before lock
[update_state Mi]After lock
User Mi is at round #4/5
[update_state Mi]After release
[update_state Lavette]Before lock
[update_state Lavette]After lock
User Lavette is at round #4/5
[update_state Lavette]After release
[update_state Mi]Before lock
[update_state Mi]After lock
User Mi is at round #4/5
[update_state Mi]After 

# File sharing initialization

In [11]:
conf = config.default
conf['n_users'] = 10
conf['n_rounds'] = 20
print(conf)

address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)
    #users[i].update_state()

u1 = users[0]
u2 = users[1]
for u in users:
    # Starts greenlet
    u.start()


print("File sender named {} has following devices: {}".format(u1.name, [d.addr for d in u1.devices]))
print("File receiver named {} has following devices: {}".format(u2.name, [d.addr for d in u2.devices]))

gevent.sleep(2)
file_info = u2.init_file_receive(u1)
# print("{} started sending file {} to {}.".format(
#     u1.name, file_info['f'].id, u2.name))


gevent.joinall(users)
print("Done")

{'header_size': 0.1, 'ack_size': 0, 'bandwidth': 10, 'period': datetime.timedelta(0, 1), 'minimum_node_availability': 0, 'n_users': 10, 'chunk_max_size': 2, 'gossip_size': 20, 'n_layers': 3, 'layer_threshold': 0.001, 'n_file_chunks': 10, 'n_rounds': 20, 'file_size': 11}
File sender named Queen has following devices: ['s17z6dhxikko', 'nyrvf7sc71jk', 'wajk9v38hc72', 'z3e7an5rxf4o', 'rf3a321phs6a', 'y17k7c6wswvl', '3trlr89ighj8', 'y3ewk44qldye']
File receiver named Demetra has following devices: ['yo9stuew5ssx', 'nxszwu6q8j6b', 'yj6yj81znbzq', 'if54awowf3fx', '5nc8hbsgh0ir', 'pja70cy9oupi', 'c37r62qzphal', 'quqc84bpiilk', 'ndcqzsrlluwi']
Device z3e7an5rxf4o (owned by Queen) will send file ni88vk
z3e7an5rxf4o (owned by Queen) successfully sent file ni88vk's chunk #3
if54awowf3fx (owned by Demetra) successfully received file ni88vk's chunk #3
z3e7an5rxf4o (owned by Queen) successfully sent file ni88vk's chunk #4
z3e7an5rxf4o (owned by Queen) successfully sent file ni88vk's chunk #2
if54awow

In [27]:
all([])

True